In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import re
import os
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torchtext.vocab import GloVe

c:\Users\Alireza\anaconda3\envs\torch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# Building the vocabulary
glove = GloVe(name='6B', dim=100, max_vectors=25000, cache = "./vectors_cache")

### Util functions

In [4]:
def load_data(csv_path, glove):

  def preprocess_string(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)

    return s

  def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
      if len(review) != 0:
        features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

  # Reading data from disk
  df = pd.read_csv(csv_path)
  # Replacing string labels with numbers
  df['sentiment'].replace({'positive': 1, 'negative': 0}, inplace=True)

  # dividing the dataset into train, validation, and test portions
  X,y = df['review'].values,df['sentiment'].values
  x_train1, x_test1, y_train, y_test = train_test_split(X, y, train_size=0.85, random_state=1)
  x_train1, x_val1, y_train, y_val = train_test_split(x_train1, y_train, train_size=0.823, random_state=1)

  print(f'Number of training examples: {len(x_train1)}')
  print(f'Number of validation examples: {len(x_val1)}')
  print(f'Number of test examples: {len(x_test1)}')


  

  x_train, x_val, x_test = [],[],[]
  for sent in x_train1:
    x_train.append([glove.stoi[preprocess_string(word)] for word in sent.lower().split() if preprocess_string(word) in glove.stoi])

  for sent in x_val1:
    x_val.append([glove.stoi[preprocess_string(word)] for word in sent.lower().split() if preprocess_string(word) in glove.stoi])

  for sent in x_test1:
    x_test.append([glove.stoi[preprocess_string(word)] for word in sent.lower().split() if preprocess_string(word) in glove.stoi])

  # Padding the data
  x_train_pad = padding_(x_train,500)
  x_val_pad = padding_(x_val,500)
  x_test_pad = padding_(x_test,500)

  # creating Tensor Datasets and DataLoaders
  train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
  valid_data = TensorDataset(torch.from_numpy(x_val_pad), torch.from_numpy(y_val))
  test_data = TensorDataset(torch.from_numpy(x_test_pad), torch.from_numpy(y_test))

  batch_size = 100

  train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
  valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)
  test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

  return train_loader, valid_loader, test_loader

def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()  
    acc = correct.sum() / len(correct)
    return acc

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def train_epoch(model, data_loader, optimizer, criterion):
  epoch_loss = 0
  epoch_acc = 0
  model.train()

  for inputs, labels in tqdm(data_loader):
      optimizer.zero_grad()
      inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float)   
      predictions = model(inputs).squeeze(1)
      loss = criterion(predictions, labels)      
      loss.backward()
      optimizer.step()
      acc = binary_accuracy(predictions, labels)
      epoch_loss += loss.item()
      epoch_acc += acc.item()
      
  return epoch_loss / len(data_loader), epoch_acc / len(data_loader)

def evaluate_epoch(model, data_loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader):
            inputs, labels = inputs.to(device), labels.to(device, dtype=torch.float)
            predictions = model(inputs).squeeze(1)
            loss = criterion(predictions, labels)
            acc = binary_accuracy(predictions, labels)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(data_loader), epoch_acc / len(data_loader)

def count_recurrent_layer_parameters(model):
    return sum(p.numel() for n, p in model.named_parameters() if p.requires_grad and 'fc' not in n)

### Loading the data

In [5]:
trainloader, validloader, testloader = load_data("IMDB Dataset.csv", glove)

Number of training examples: 34977
Number of validation examples: 7523
Number of test examples: 7500


# Q0 - 2 Level Bi-LSTM
#### (The example model)

In [5]:
class LSTMBi2Level(nn.Module):
  def __init__(self, hidden_dim, embed_dim):
    super(LSTMBi2Level, self).__init__()
    
    self.hidden_dim = hidden_dim
    self.embed_dim = embed_dim

    self.embedding = nn.Embedding.from_pretrained(glove.vectors, freeze=True)

    self.lstm_bi_2_level = nn.LSTM(
      input_size=self.embed_dim,
      hidden_size=self.hidden_dim,
      bidirectional=True,
      num_layers=2,
      batch_first=True
    )

    self.fc = nn.Linear(self.hidden_dim * 2, 1)

  def forward(self, x):
    embeds = self.embedding(x)
    output, (hidden, cell) = self.lstm_bi_2_level(embeds)

    # hidden shape = [n_directions * n_layers, batch_size, hidden_dim]
    # Concatinate the last forward layer, and the last backward layer
    hidden = torch.cat((hidden[2, :, :], hidden[3, :, :]), dim=1)

    logit = self.fc(hidden)
    return logit

In [6]:
model = LSTMBi2Level(embed_dim=100, hidden_dim=200).to(device)

In [9]:
print(f'The 2 Level Bidirectional LSTM model has {count_recurrent_layer_parameters(model):,} parameters in the four bi-LSTM layers')

The 2 Level Bidirectional LSTM model has 1,446,400 parameters in the four bi-LSTM layers


#### Training the model and evaluating the validation data per epoch

In [26]:
N_EPOCHS = 5

optimizer = Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss().to(device)

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train_epoch(model, trainloader, optimizer, criterion)
    valid_loss, valid_acc = evaluate_epoch(model, validloader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        os.makedirs('bi-lstm-2-level', exist_ok=True)
        torch.save(model.state_dict(), 'bi-lstm-2-level/best-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.5f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.5f} |  Val. Acc: {valid_acc*100:.2f}%')

100%|██████████| 76/76 [00:06<00:00, 12.16it/s]


Epoch: 01 | Epoch Time: 1m 39s
	Train Loss: 0.48669 | Train Acc: 76.58%
	 Val. Loss: 0.39203 |  Val. Acc: 82.89%


100%|██████████| 76/76 [00:06<00:00, 12.34it/s]


Epoch: 02 | Epoch Time: 1m 41s
	Train Loss: 0.35195 | Train Acc: 84.49%
	 Val. Loss: 0.32429 |  Val. Acc: 85.80%


100%|██████████| 76/76 [00:06<00:00, 12.43it/s]


Epoch: 03 | Epoch Time: 1m 41s
	Train Loss: 0.30624 | Train Acc: 86.83%
	 Val. Loss: 0.29141 |  Val. Acc: 87.51%


100%|██████████| 76/76 [00:06<00:00, 12.02it/s]


Epoch: 04 | Epoch Time: 1m 40s
	Train Loss: 0.27796 | Train Acc: 88.31%
	 Val. Loss: 0.29145 |  Val. Acc: 87.57%


100%|██████████| 76/76 [00:06<00:00, 12.36it/s]

Epoch: 05 | Epoch Time: 1m 39s
	Train Loss: 0.25576 | Train Acc: 89.39%
	 Val. Loss: 0.27565 |  Val. Acc: 88.32%


#### Evaluate the model on testing data

In [29]:
test_loss, test_acc = evaluate_epoch(model, testloader, criterion)
print(f'Test accuracy of the 2 Level Bidirectional LSTM is {test_acc}')

100%|██████████| 75/75 [00:06<00:00, 11.86it/s]

Test accuracy of the 2 Level Bidirectional LSTM is 0.8865333135922749
